Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Handle cases where no transit route is found

TODO: Push resulting data into pandas dataframe



In [2]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [8]:
def print_transit(query_json):
    """This function takes a json response from an api call to OTP and prints important variables about a trip"""
    for a in query_json['plan']['itineraries']:
        print('start: '+ tclean(a['startTime']))
        print('end: '+ tclean(a['endTime']))
        print('duration: '+ str(a['duration']/60) + ' minutes')
        print('waitingTime: '+ str(a['waitingTime']/60) + ' minutes')
        print('transitTime: '+ str(a['transitTime']/60) + ' minutes')
        print('walkDistance: '+ str(a['walkDistance']/1000*.621371) + ' miles')
        print('transfers: '+ str(a['transfers']))
        for each in a['legs']:
            print('   ' + each['from']['name'])
            print('   StartTime: '+ tclean(each['startTime']) + " mode: " + each['mode'])
            print("   headsign: " + str(each.get('headsign')))
            print('   endTime:   ' + tclean(each['endTime']) + " duration: " + str(each['duration']/60))    

In [9]:
#rand_starts=pd.read_csv('Rand_Starts.csv')
#rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')
#known_starts=pd.read_csv('Hidden_Starts.csv')
#known_dests=pd.read_csv('Hidden_Dests.csv')

df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [10]:
def tclean(dto):
    """converts unix milliseconds to times"""
    return str(datetime.fromtimestamp(dto/1000))

In [11]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose
2,37.400787,-121.868619,1684 Garvey Pl,37.400281,-121.939691,3331 N. 1st St VTA
3,37.400281,-121.939691,3331 N. 1st St VTA,37.333539,-121.885089,"116 Paseo De San Antonio Walk, San Jose, CA 95..."
4,37.397893,-121.868478,"2755 Lucena Dr, San Jose, CA 95132, USA",37.400281,-121.939691,3331 N. 1st St VTA


In [12]:
#https://github.com/opentripplanner/OpenTripPlanner/blob/257ac13150b40942fdae4812007545a66546d7da/src/main/java/org/opentripplanner/routing/core/RoutingRequest.java
row = 0
payload = {}
payload.update({'fromPlace': '%s,%s' % (df['start_lat'][row],df['start_long'][row])})
payload.update({'toPlace': '%s, %s' % (df['dest_lat'][row], + df['dest_long'][row])})
payload.update({'mode':'WALK,BUSISH,TRAINISH'})
payload.update({'date':'2016-07-14'})
payload.update({'time':'07:00'})
payload.update({'maxTransfers':3,'optimize':'QUICK'})
payload.update({'numItineraries':5})
payload.update({'maxWalkDistance':5000})
payload.update({'bikeSpeed':5})
payload.update({'arriveBy':'false'})
payload.update({'waitAtBeginning':5.95})
payload.update({'walkReluctance':12})
payload.update({'maxPreTransitTime':100})

r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
#r.text
print_transit(r.json())

start: 2016-07-14 07:00:47
end: 2016-07-14 07:23:19
duration: 22.533333333333335 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 17.0 minutes
walkDistance: 0.25768267984135684 miles
transfers: 0
   Start point 0.7534454429901686
   StartTime: 2016-07-14 07:00:47 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:01:59 duration: 1.2
   HOSTETTER STATION (1)
   StartTime: 2016-07-14 07:02:00 mode: TRAM
   headsign: SANTA TERESA
   endTime:   2016-07-14 07:19:00 duration: 17.0
   RIVER OAKS STATION (1)
   StartTime: 2016-07-14 07:19:01 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:23:19 duration: 4.3
start: 2016-07-14 07:15:47
end: 2016-07-14 07:38:19
duration: 22.533333333333335 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 17.0 minutes
walkDistance: 0.25768267984135684 miles
transfers: 0
   Start point 0.7534454429901686
   StartTime: 2016-07-14 07:15:47 mode: WALK
   headsign: None
   endTime:   2016-07-14 07:16:59 duration: 1.2
   HOSTETT

In [14]:
#for each in range(1,59):
    #print(each)
row = 4
payload = {}

#walkReluctance=[0,10,20,100]
#time = [1,2,3,4,5]
#maxWalk
payload.update({'date':'2016-07-15', 'time':'07:00', 'maxTransfers':3,'optimize':'QUICK'})
payload.update({'fromPlace': '%s,%s' % (df['start_lat'][row],df['start_long'][row])})
payload.update({'toPlace': '%s, %s' % (df['dest_lat'][row], + df['dest_long'][row])})
#https://github.com/opentripplanner/OpenTripPlanner/blob/257ac13150b40942fdae4812007545a66546d7da/src/main/java/org/opentripplanner/routing/core/TraverseModeSet.java
#payload.update({'mode':'WALK,TRAINISH'})
#payload.update({'mode':'WALK,TRAINISH,RAIL,CABLE_CAR'})
#payload.update({'mode':'WALK,TRAM'})
#payload.update({'mode':'WALK,TRAM, BUSISH'})
payload.update({'mode':'BICYCLE,TRANSIT'})
#payload.update({'mode':'CAR'})
#payload.update({'mode':'WALK,TRANSIT'})
#payload.update({'walkReluctance':20})
payload.update({'walkReluctance':20})
#payload.update({'waitReluctance':0})
payload.update({'time':'07:00'})
#payload.update({'time':'07:%s' % (each)})
#payload.update({'walkBoardCost':500})
#payload.update({'bikeBoardCost':0})
payload.update({'maxTranfers':2})
#payload.update({'preTransitPenalty':})
#payload.update({'waitAtBeginning':100000000})
#payload.update({'maxPreTransitTime':10000})
payload.update({'maxWalkDistance':5000})
payload.update({'numItineraries':5})
#payload.update({'softWalkPenalty':60.0})
#payload.update({'softWalkOverageRate':5.0})
r = requests.get('http://vta2.amigocloud.com/api/otp/plan?', params=payload)
#r.text
print_transit(r.json())

start: 2016-07-15 07:14:26
end: 2016-07-15 07:35:42
duration: 21.266666666666666 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 17.0 minutes
walkDistance: 0.7153594531857999 miles
transfers: 0
   Start point 0.6608056659763244
   StartTime: 2016-07-15 07:14:26 mode: BICYCLE
   headsign: None
   endTime:   2016-07-15 07:16:59 duration: 2.55
   HOSTETTER STATION (1)
   StartTime: 2016-07-15 07:17:00 mode: TRAM
   headsign: SANTA TERESA
   endTime:   2016-07-15 07:34:00 duration: 17.0
   RIVER OAKS STATION (1)
   StartTime: 2016-07-15 07:34:01 mode: BICYCLE
   headsign: None
   endTime:   2016-07-15 07:35:42 duration: 1.6833333333333333
start: 2016-07-15 07:28:26
end: 2016-07-15 07:50:42
duration: 22.266666666666666 minutes
waitingTime: 0.03333333333333333 minutes
transitTime: 18.0 minutes
walkDistance: 0.7153594531857999 miles
transfers: 0
   Start point 0.6608056659763244
   StartTime: 2016-07-15 07:28:26 mode: BICYCLE
   headsign: None
   endTime:   2016-07-15 07:30:59 d

In [ ]:
#hidden starts/deskt
#row 3
#Transit + walk fails to take transit

In [ ]:
r.url

In [ ]:
df.head()
#Random From and To pints

In [ ]:
, '.join([str(x) for x in list])
''.join(map(str,list))

In [ ]:
walkReluctance=[0,10,20,100]
time = [1,2,3,4,5]
cartesian((walkReluctance,time))


In [ ]:
import numpy as np

def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.

    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])

    """
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

walkReluctance=[0,10,20,100]
time = [1,2,3,4,5]
waitAtBeginning = [1,10,50,100,1000,2000,5000,10000,100000]
array = cartesian((walkReluctance,time,waitAtBeginning))